In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

from fastai.learner import *

import torchtext
from torchtext import vocab, data
from torchtext.datasets import language_modeling

from fastai.rnn_reg import *
from fastai.rnn_train import *
from fastai.nlp import *
from fastai.lm_rnn import *

import dill as pickle
import pandas

## Load Data

In [2]:
PATH ='data/'

In [3]:
train = pd.read_csv('data/kaggle/train.csv')
test = pd.read_csv('data/kaggle/test.csv')
sample = pd.read_csv('data/kaggle/sample_submission.csv')

In [4]:
train._get_numeric_data().mean()

toxic            0.095844
severe_toxic     0.009996
obscene          0.052948
threat           0.002996
insult           0.049364
identity_hate    0.008805
dtype: float64

The training class is highly unbalanced, need to pay attention to this.

## Data Processing (Fastai library)

In [18]:
bs =64; bptt = 70

In [20]:
??LanguageModelData.from_dataframes

In [21]:
TEXT = data.Field(lower=True, tokenize=spacy_tok)

In [23]:
train_test=train.head(2)

In [25]:
md = LanguageModelData.from_dataframes(PATH,TEXT,'comment_text',train_test,train_test,train_test)

AssertionError: Torch not compiled with CUDA enabled

## Data Processing

In [5]:
TEXT = data.Field(lower= True,tokenize=spacy_en)

In [6]:
bs=64; bptt=70

In [7]:
train.loc[3,'comment_text']

'"\nMore\nI can\'t make any real suggestions on improvement - I wondered if the section statistics should be later on, or a subsection of ""types of accidents""  -I think the references may need tidying so that they are all in the exact same format ie date format etc. I can do that later on, if no-one else does first - if you have any preferences for formatting style on references or want to do it yourself please let me know.\n\nThere appears to be a backlog on articles for review so I guess there may be a delay until a reviewer turns up. It\'s listed in the relevant form eg Wikipedia:Good_article_nominations#Transport  "'

In [17]:
train.comment_text.str.replace("\n", " ")

0         Explanation Why the edits made under my userna...
1         D'aww! He matches this background colour I'm s...
2         Hey man, I'm really not trying to edit war. It...
3         " More I can't make any real suggestions on im...
4         You, sir, are my hero. Any chance you remember...
5         "  Congratulations from me as well, use the to...
6              COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK
7         Your vandalism to the Matt Shirvington article...
8         Sorry if the word 'nonsense' was offensive to ...
9         alignment on this subject and which are contra...
10        " Fair use rationale for Image:Wonju.jpg  Than...
11        bbq   be a man and lets discuss it-maybe over ...
12        Hey... what is it.. @ | talk . What is it... a...
13        Before you start throwing accusations and warn...
14        Oh, and the girl above started her arguments w...
15        "  Juelz Santanas Age  In 2002, Juelz Santana ...
16        Bye!   Don't look, come or thi

## Build Model

## Train model